## 0. 读入原始的文本
说明：  
文件中，中国和科学院之间添加了一个空格，是因为，如果不加空格，jieba会把中国科学院分词为一个词，和演练的word中不一致了。 
故此，在中国和科学院之间，添加一个空格，目的仅仅是因为保证和演练的word文档中的执行效果一致。  
实际工作中，不用这样子。

In [1]:
# 只读方式打开utf-8编码的文件
with open("sentences.txt", encoding='utf-8', mode='r') as f:
    # 读入文件内容到 text 中
    text = f.read()
    text
# 按照换行符切分文本为语句列表
sentences = text.splitlines()
sentences

'我来到北京清华大学\n他来到了网易杭研大厦\n小明硕士毕业与中国 科学院\n我爱北京天安门'

['我来到北京清华大学', '他来到了网易杭研大厦', '小明硕士毕业与中国 科学院', '我爱北京天安门']

## 1. 使用结巴分词对每个语句进行分词（采用精确模式）
参考： https://zhuanlan.zhihu.com/p/29747350 

In [2]:
# 导入 jieba
# 可能需要安装 jieba 第三方包，可以在 jupyter 的单元格中，运行： ! pip install jieba
import jieba
# 声明空列表
corpus = []
# 遍历语句列表中的每一条语句
for sentence in sentences:
    # 显示每一条语句内容
    sentence
    # 使用精确模式对一条语句进行分词，返回的是含有分词结果的迭代器
    wordlist_iterator=jieba.cut(sentence)
    # 将分词结果迭代器中的每个单词后面添加空格，生成一条语句的分词列表，此处是字符串类型
    wordlist = ' '.join(wordlist_iterator)
    # 将语句分词后的追加空格间隔的分词字符串追加到 corpus 列表中
    corpus.append(wordlist)
# 输出 语句分词字符串列表
corpus

# # 自定义文本
# corpus = ["我 来到 北京 清华大学",  # 第一类文本切词后的结果，词之间以空格隔开  
#           "他 来到 了 网易 杭研 大厦",  # 第二类文本的切词结果  
#           "小明 硕士 毕业 与 中国 科学院",  # 第三类文本的切词结果  
#           "我 爱 北京 天安门"]  # 第四类文本的切词结果  

'我来到北京清华大学'

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\txsli\AppData\Local\Temp\jieba.cache
Loading model cost 0.763 seconds.
Prefix dict has been built successfully.


'他来到了网易杭研大厦'

'小明硕士毕业与中国 科学院'

'我爱北京天安门'

['我 来到 北京 清华大学', '他 来到 了 网易 杭研 大厦', '小明 硕士 毕业 与 中国   科学院', '我 爱 北京 天安门']

## 2. 计算词频矩阵（使用哈工大停用词表）

In [3]:
# 导入所需的包
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.feature_extraction.text import CountVectorizer 

### 2.1 获取停用词表
Python文本分析-常用中文停用词表（Chinese Stop Words） https://blog.csdn.net/purpen/article/details/105468646   
最全中文停用词表（可直接复制） https://blog.csdn.net/dilifish/article/details/117885706

In [4]:
# 读入哈工大停用词表文件
stopword_filename = "hit_stopwords.txt"
# 打开停用词文件指针，指定编码（支持中文）和模式（读入模式）
stopword_file = open(stopword_filename, encoding='utf-8', mode='r')
# 读入停用词文件的内容
stopword_content = stopword_file.read()
# 关闭文件指针
stopword_file.close()
# 输出停用词表文件的内容
#stopword_content
# 按照换行转换为停用词列表
stopword_list = stopword_content.splitlines()
# 输出停用词列表
#stopword_list

### 2.2 生成词频矩阵（其实是词袋（word-vec）模型）
CountVectorizer介绍 https://zhuanlan.zhihu.com/p/37644086  
https://www.qqxiuzi.cn/bianma/zifuji.php  
本质：  
去掉所有句子中的停用词，多个句子中重复出现的单词只保留一次，  
然后按首字的unicode编码顺序排序为一个向量（feature_names）  
将所有句子中剩下的非停用词，生成词频矩阵（4行12列的稀疏矩阵），规则为：  
行坐标为语句序号，从0开始，本例为4句话，故为： 0到3  
列坐标是本行语句中非停用词对应向量（feature_names）的位置（从0开始，本例共12个唯一的非停用词，故为：0到11），如果该位置有本句的单词则值为1，否则为0。  
故此是稀疏矩阵，并用稀疏矩阵的表示形式输出。  
**可以这么理解，本例中，一个句子就是一篇文章，所有的四个句子就是所有的文章。**

In [5]:
# 将文本中的词语转换为词频矩阵
# 将文本中的词语转换为词频矩阵，矩阵元素a[i][j] 表示j词在i类文本下的词频 
# 创建词袋数据结构
# 此处指定使用停用词列表作为停词表
# 也可以不使用，而是参考max_df设置，默认为1，以根据术语内部语料库文档频率自动检测和过滤停用词。
vectorizer=CountVectorizer(stop_words=stopword_list) 
# 对 语句分词字符串列表-corpus 进行训练和转换，生成词频矩阵
vectors=vectorizer.fit_transform(corpus)
# 输出 词频矩阵的数据结构
vectors
# 输出 语句分词字符串列表 中的唯一词，即词频矩阵中所有出现的那些的唯一的单词
# vectorizer.get_feature_names()  # 已经降级，使用 get_feature_names_out 替换
vectorizer.get_feature_names_out()
# 稀疏矩阵的输出方式：
print(vectors)
# 原始矩阵的输出方式：
print(vectors.toarray())

C:\Dev\Python\venvs\work\lib\site-packages\sklearn\feature_extraction\text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['exp', 'lex', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｆ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', '１２', 'ｌｉ', 'ｚｘｆｉｔｌ'] not in stop_words.
  warnings.warn(


<4x12 sparse matrix of type '<class 'numpy.int64'>'
	with 14 stored elements in Compressed Sparse Row format>

array(['中国', '北京', '大厦', '天安门', '小明', '来到', '杭研', '毕业', '清华大学', '硕士',
       '科学院', '网易'], dtype=object)

  (0, 5)	1
  (0, 1)	1
  (0, 8)	1
  (1, 5)	1
  (1, 11)	1
  (1, 6)	1
  (1, 2)	1
  (2, 4)	1
  (2, 9)	1
  (2, 7)	1
  (2, 0)	1
  (2, 10)	1
  (3, 1)	1
  (3, 3)	1
[[0 1 0 0 0 1 0 0 1 0 0 0]
 [0 0 1 0 0 1 1 0 0 0 0 1]
 [1 0 0 0 1 0 0 1 0 1 1 0]
 [0 1 0 1 0 0 0 0 0 0 0 0]]


## 3. 统计每个词语的tf-idf权值
参考：  https://zhuanlan.zhihu.com/p/166636681 、 https://www.ruanyifeng.com/blog/2013/03/tf-idf.html  
tf：词频，某个词语在整篇文章中出现次数，一般除以整篇文章的词语总数，让tf在（0~1）之间，即所谓的归一化。  
idf：逆词频，所有文章中包含某个词语的频率，用所有文章数除以包含这个词语的文章数表征，一般使用公式：log(总文章数/含有某个词语的文章数+1)。其中：  
分母+1，防止没有包含某词语的文章的情况，此时含有某个词语的文章数为0，为了保证分母不能为0，故+1  
使用log函数，一般是借助log函数的单调增函数的性质，缩小结果数据的绝对数值（参考：https://zhuanlan.zhihu.com/p/106232513 ），因为本例中总文章只有四篇，但是在实际应用中，总文章数应该是现实世界中的所有文章（能找到的），数量会很大，使用log主要的目的就是降低结果数据的值。
**想想稀疏矩阵的性质，按行、列求和，都是是什么意思？**

In [6]:
# 该类会统计每个词语的tf-idf权值
transformer = TfidfTransformer()  
tfidf = transformer.fit_transform(vectors)
# 输出tfidf的类型
tfidf
# 输出tfidf的稀疏矩阵形式，并按每篇文章（每句话）中的每个词语的tf-idf的权值逆序排序
print(tfidf)
print(tfidf.toarray())

<4x12 sparse matrix of type '<class 'numpy.float64'>'
	with 14 stored elements in Compressed Sparse Row format>

  (0, 8)	0.6676785446095399
  (0, 5)	0.5264054336099155
  (0, 1)	0.5264054336099155
  (1, 11)	0.5254727492640658
  (1, 6)	0.5254727492640658
  (1, 5)	0.41428875116588965
  (1, 2)	0.5254727492640658
  (2, 10)	0.4472135954999579
  (2, 9)	0.4472135954999579
  (2, 7)	0.4472135954999579
  (2, 4)	0.4472135954999579
  (2, 0)	0.4472135954999579
  (3, 3)	0.7852882757103967
  (3, 1)	0.6191302964899972
[[0.         0.52640543 0.         0.         0.         0.52640543
  0.         0.         0.66767854 0.         0.         0.        ]
 [0.         0.         0.52547275 0.         0.         0.41428875
  0.52547275 0.         0.         0.         0.         0.52547275]
 [0.4472136  0.         0.         0.         0.4472136  0.
  0.         0.4472136  0.         0.4472136  0.4472136  0.        ]
 [0.         0.6191303  0.         0.78528828 0.         0.
  0.         0.         0.         0.         0.         0.        ]]


## 4. 输出相关词频、权重信息

### 4.1 输出所有的非停用词的词语

In [7]:
# 获取词袋模型(vectorizer)中的所有词语
# word = vectorizer.get_feature_names_out()
word = vectorizer.get_feature_names()  # 这个看的更清晰
word

C:\Dev\Python\venvs\work\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['中国', '北京', '大厦', '天安门', '小明', '来到', '杭研', '毕业', '清华大学', '硕士', '科学院', '网易']

In [8]:
# 将tf-idf矩阵抽取出来，元素a[i][j]（即：一篇文章中某个词语）表示j词在i类文本中的tf-idf权重
weight = tfidf.toarray() 
weight

array([[0.        , 0.52640543, 0.        , 0.        , 0.        ,
        0.52640543, 0.        , 0.        , 0.66767854, 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.52547275, 0.        , 0.        ,
        0.41428875, 0.52547275, 0.        , 0.        , 0.        ,
        0.        , 0.52547275],
       [0.4472136 , 0.        , 0.        , 0.        , 0.4472136 ,
        0.        , 0.        , 0.4472136 , 0.        , 0.4472136 ,
        0.4472136 , 0.        ],
       [0.        , 0.6191303 , 0.        , 0.78528828, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ]])

In [9]:
# 打印每篇文章中每个单词的tf-idf权重
# 第一个for遍历所有文本，第二个for遍历某一类文本下的词语权重
for i in range(len(weight)):  
    print("-------这里输出第", i, "类文本的词语tf-idf权重------")  
    for j in range(len(word)):  
        print(word[j], weight[i][j])  

-------这里输出第 0 类文本的词语tf-idf权重------
中国 0.0
北京 0.5264054336099155
大厦 0.0
天安门 0.0
小明 0.0
来到 0.5264054336099155
杭研 0.0
毕业 0.0
清华大学 0.6676785446095399
硕士 0.0
科学院 0.0
网易 0.0
-------这里输出第 1 类文本的词语tf-idf权重------
中国 0.0
北京 0.0
大厦 0.5254727492640658
天安门 0.0
小明 0.0
来到 0.41428875116588965
杭研 0.5254727492640658
毕业 0.0
清华大学 0.0
硕士 0.0
科学院 0.0
网易 0.5254727492640658
-------这里输出第 2 类文本的词语tf-idf权重------
中国 0.4472135954999579
北京 0.0
大厦 0.0
天安门 0.0
小明 0.4472135954999579
来到 0.0
杭研 0.0
毕业 0.4472135954999579
清华大学 0.0
硕士 0.4472135954999579
科学院 0.4472135954999579
网易 0.0
-------这里输出第 3 类文本的词语tf-idf权重------
中国 0.0
北京 0.6191302964899972
大厦 0.0
天安门 0.7852882757103967
小明 0.0
来到 0.0
杭研 0.0
毕业 0.0
清华大学 0.0
硕士 0.0
科学院 0.0
网易 0.0


# 以上！